# Импорты и пути к моделям

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.auto import tqdm

import pandas as pd

import lightning.pytorch as pl

from utils import get_name_labse_embs, text_preprocess

import json
from nltk.tokenize import RegexpTokenizer
import gc
from thefuzz import fuzz
import numpy as np
from tqdm.auto import tqdm
from catboost import CatBoostClassifier, Pool
import warnings

from torch.utils.data import Dataset

from lightning.pytorch.callbacks import ModelCheckpoint

from torch import nn

from sklearn.metrics import roc_auc_score

In [3]:
PATH_TO_LABSE = "./models/LaBSE.pt"
PATH_TO_MULTIMODAL = "./models/Multi.pt"

In [ ]:
pl.seed_everything(56, workers=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Получаем эмбединги названий LaBSE

In [4]:
named_data = pd.read_parquet('./datasets/train_data.parquet', columns=["variantid", "name"])

In [5]:
named_data.head()

,variantid,name
0,51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,..."
1,53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...
2,56763357,"Набор микропрепаратов Konus 25: ""Клетки и ткан..."
3,56961772,"Мобильный телефон BQ 1848 Step, черный"
4,61054740,"Штатив трипод Tripod 330A для фотоаппаратов, в..."


In [ ]:
name_labse_768 = get_name_labse_embs("cointegrated/LaBSE-en-ru", sentences=list(named_data["name"]), device=device)

# Получаем эмбединги от LaBSE Tuned

In [ ]:
class Args:
    batch_size = 96
    epochs = 5
    lr = 1e-5
    lr_warmup_epochs = 5
    lr_warmup_decay = 0.01
    lr_min = 1e-5

args = Args()

In [ ]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.pairs_len = len(self.pairs)

        self.names = data['name'].apply(text_preprocess)

    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            self.names[id1],
            self.names[id2],
            target
        )

In [ ]:
class LaBSE(pl.LightningModule):
    margin = 0.75

    def __init__(self):
        super(LaBSE, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained('cointegrated/LaBSE-en-ru')
        self.model = AutoModel.from_pretrained('cointegrated/LaBSE-en-ru')

        self.fc = nn.Linear(768, 768)

        #for param in self.model.embeddings.parameters():
        #    param.requires_grad = False
        #for param in self.model.encoder.parameters():
        #    param.requires_grad = False

    def forward(self, x):
        encoded_input = self.tokenizer(x, padding=True, truncation=True, max_length=256, return_tensors='pt').to('cuda')
        model_output = self.model(**encoded_input)

        embeddings = torch.nn.functional.normalize(model_output.pooler_output)
        embeddings = self.fc(embeddings)
        return embeddings

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=0.05
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        # self.log('step', batch_idx, logger=True, on_epoch=True)
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)

        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, on_step=False, logger=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)

        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=False, on_epoch=True, prog_bar=True)

        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0

        self.log("val_auc", auc, logger=False, on_epoch=True, prog_bar=True)

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

    def predict_step(self, batch, batch_idx):
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)

        dists = nn.PairwiseDistance()(out1, out2)
        return torch.cat([out1, out2, (1 - dists).unsqueeze(-1)], dim=1).detach().cpu()

In [ ]:
model = LaBSE()

In [ ]:
model.load_state_dict(torch.load(PATH_TO_LABSE, map_location=torch.device('cpu')))

In [ ]:
trainer = pl.Trainer(
    logger=False, # CSVLogger('./'),
    enable_checkpointing=False,

    accelerator='gpu',
    devices=[0],
    profiler='advanced',
    precision="16-mixed",
    check_val_every_n_epoch=1,
    max_epochs=args.epochs
)

In [ ]:
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=['variantid', 'name']).set_index('variantid')

In [ ]:
test_pairs['target'] = -1
test_pairs = test_pairs[['target', 'variantid1', 'variantid2']]
test_dataset = ItemsDataset(test_pairs, test_data)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    num_workers=17,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [ ]:
test_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, test_loader)])

In [ ]:
test_embeds = pd.Series(index=test_data.index, dtype='object', name='labse_tuned_768')
test_embeds[test_pairs.variantid1] = list(test_features[:, :768])
test_embeds[test_pairs.variantid2] = list(test_features[:, 768:768*2])
test_embeds

# Получаем эмбединги от мультимодальной сети

In [ ]:
test_data = pd.read_parquet('./datasets/test_data.parquet').set_index('variantid')
test_data['categories'] = test_data['categories'].apply(lambda x: json.loads(x))
test_data['main_pic_embeddings_resnet_v1'] = test_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
test_data

In [ ]:
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
test_pairs

In [ ]:
test_cat3 = set()
for categories in test_data.categories:
    test_cat3.add(categories['3'])

In [ ]:
colors_mapper = {
 'ярко-синий': 'ярко-синий',
 'ярко-розовый': 'ярко-розовый',
 'ярко-зеленый': 'ярко-зеленый',
 'ярко-желтый': 'ярко-желтый',
 'янтарный': 'янтарный',
 'электрик': 'электрик',
 'экрю': 'экрю',
 'шоколадный': 'шоколадный',
 'черный': 'черный',
 'черно-синий': 'черно-синий',
 'черно-серый': 'черно-серый',
 'черно-красный': 'черно-красный',
 'черно-зеленый': 'черно-зеленый',
 'черн': 'черный',
 'чер': 'черный',
 'циан': 'бирюзовый',
 'цементный': 'цементный',
 'хаки': 'хаки',
 'фуксия': 'фуксия',
 'фисташковый': 'фисташковый',
 'фиолетовый': 'фиолетовый',
 'фиолетово-синий': 'фиолетово-синий',
 'фиолет': 'фиолетовый',
 'фиол': 'фиолетовый',
 'фиалковый': 'фиалковый',
 'тыквенный': 'тыквенный',
 'тыква': 'тыквенный',
 'травяной': 'травяной',
 'томатный': 'томатный',
 'тиффани': 'тиффани',
 'терракотовый': 'терракотовый',
 'терракота': 'терракотовый',
 'темно-фиолетовый': 'темно-фиолетовый',
 'темно-синий': 'темно-синий',
 'темно-серый': 'темно-серый',
 'темно-розовый': 'темно-розовый',
 'темно-оранжевый': 'темно-оранжевый',
 'темно-оливковый': 'темно-оливковый',
 'темно-красный': 'темно-красный',
 'темно-коричневый': 'темно-коричневый',
 'темно-зеленый': 'темно-зеленый',
 'темно-голубой': 'темно-голубой',
 'темно-бирюзовый': 'темно-бирюзовый',
 'темно-бежевый': 'темно-бежевый',
 'сливовый': 'сливовый',
 'сиреневый': 'сиреневый',
 'синий': 'синий',
 'сине-зеленый': 'сине-зеленый',
 'син': 'синий',
 'серый': 'серый',
 'серовато-зеленый': 'серовато-зеленый',
 'серо-коричневый': 'серо-коричневый',
 'серо-зеленый': 'серо-зеленый',
 'серо-голубой': 'серо-голубой',
 'серо-бежевый': 'серо-бежевый',
 'серебряный': 'серебряный',
 'серебристый': 'серебристый',
 'серебристо-серый': 'серебристо-серый',
 'сер': 'серый',
 'сепия': 'сепия',
 'светло-фиолетовый': 'светло-фиолетовый',
 'светло-синий': 'светло-синий',
 'светло-серый': 'светло-серый',
 'светло-розовый': 'светло-розовый',
 'светло-пурпурный': 'светло-пурпурный',
 'светло-коричневый': 'светло-коричневый',
 'светло-золотистый': 'светло-золотистый',
 'светло-зеленый': 'светло-зеленый',
 'светло-желтый': 'светло-желтый',
 'светло-голубой': 'светло-голубой',
 'светло-бирюзовый': 'светло-бирюзовый',
 'светло-бежевый': 'светло-бежевый',
 'сапфировый': 'сапфировый',
 'салатовый': 'салатовый',
 'рыжий': 'рыжий',
 'розовый': 'розовый',
 'розово-фиолетовый': 'розово-фиолетовый',
 'розово-золотой': 'розово-золотой',
 'разноцветный': 'разноцветный',
 'пурпурный': 'пурпурный',
 'пурпурно-фиолетовый': 'пурпурно-фиолетовый',
 'песочный': 'песочный',
 'перу': 'перу',
 'персиковый': 'персиковый',
 'охра': 'охра',
 'орхидея': 'орхидея',
 'оранжевый': 'оранжевый',
 'оранжево-розовый': 'оранжево-розовый',
 'оливковый': 'оливковый',
 'огненно-красный': 'огненно-красный',
 'нефритовый': 'нефритовый',
 'небесный': 'небесный',
 'мятный': 'мятный',
 'мятно-зеленый': 'мятно-зеленый',
 'мята': 'мятный',
 'мультиколор': 'мультиколор',
 'морковный': 'морковный',
 'молочный': 'молочный',
 'многоцветный': 'многоцветный',
 'медный': 'медный',
 'марсала': 'марсала',
 'малиновый': 'малиновый',
 'малиново-красный': 'малиново-красный',
 'малахитовый': 'малахитовый',
 'льняной': 'льняной',
 'лимонный': 'лимонный',
 'лиловый': 'лиловый',
 'латунный': 'латунный',
 'лаймовый': 'лаймовый',
 'лайм': 'лаймовый',
 'лазурный': 'лазурный',
 'лавандовый': 'лавандовый',
 'лаванда': 'лавандовый',
 'кремовый': 'кремовый',
 'красный': 'красный',
 'красновато-коричневый': 'красновато-коричневый',
 'красно-оранжевый': 'красно-оранжевый',
 'красно-коричневый': 'красно-коричневый',
 'красн': 'красный',
 'крас': 'красный',
 'кофейный': 'кофейный',
 'космос': 'космос',
 'коричневый': 'коричневый',
 'коричнево-красный': 'коричнево-красный',
 'коричнево-бежевый': 'коричнево-бежевый',
 'коралловый': 'коралловый',
 'кораллово-красный': 'кораллово-красный',
 'кобальтовый': 'кобальтовый',
 'кирпичный': 'кирпичный',
 'кирпично-красный': 'кирпично-красный',
 'кварцевый': 'кварцевый',
 'кардинал': 'кардинал',
 'канареечный': 'канареечный',
 'камуфляжный': 'камуфляжный',
 'индиго': 'индиго',
 'изумрудный': 'изумрудный',
 'изумрудно-зеленый': 'изумрудно-зеленый',
 'изумруд': 'изумрудный',
 'золотой': 'золотой',
 'золотистый': 'золотистый',
 'зеленый': 'зеленый',
 'зелено-серый': 'зелено-серый',
 'зел': 'зеленый',
 'жемчужно-белый': 'жемчужно-белый',
 'желтый': 'желтый',
 'желто-розовый': 'желто-розовый',
 'желто-зеленый': 'желто-зеленый',
 'желт': 'желтый',
 'гусеница': 'гусеница',
 'грушевый': 'грушевый',
 'графит': 'графит',
 'гранитный': 'гранитный',
 'гранатовый': 'гранатовый',
 'горчичный': 'горчичный',
 'голубой': 'голубой',
 'голуб': 'голубой',
 'глициния': 'глициния',
 'вишня': 'вишневый',
 'вишневый': 'вишневый',
 'васильковый': 'васильковый',
 'ванильный': 'ванильный',
 'бурый': 'бурый',
 'бронзовый': 'бронзовый',
 'бордовый': 'бордовый',
 'бордо': 'бордовый',
 'болотный': 'болотный',
 'бледно-розовый': 'бледно-розовый',
 'бледно-пурпурный': 'бледно-пурпурный',
 'бледно-желтый': 'бледно-желтый',
 'бирюзовый': 'бирюзовый',
 'бирюзово-зеленый': 'бирюзово-зеленый',
 'белый': 'белый',
 'белоснежный': 'белоснежный',
 'бело-зеленый': 'бело-зеленый',
 'бел': 'белый',
 'бежевый': 'бежевый',
 'бежево-серый': 'бежево-серый',
 'бежево-розовый': 'бежево-розовый',
 'баклажановый': 'баклажановый',
 'антрацитовый': 'антрацитовый',
 'аметистовый': 'аметистовый',
 'алый': 'алый',
 'аквамариновый': 'аквамариновый',
 'аква': 'аква',
 'абрикосовый': 'абрикосовый',
 'yellow': 'желтый',
 'wine': 'wine',
 'white': 'белый',
 'violet': 'фиолетовый',
 'vanilla': 'ванильный',
 'ultramarine': 'ultramarine',
 'turquoise': 'бирюзовый',
 'tomato': 'томатный',
 'teal': 'teal',
 'tan': 'tan',
 'snow': 'snow',
 'silver': 'серебряный',
 'sapphire': 'сапфировый',
 'red': 'красный',
 'purple': 'фиолетовый',
 'pink': 'розовый',
 'peru': 'перу',
 'pear': 'грушевый',
 'peach': 'персиковый',
 'orchid': 'орхидея',
 'orange': 'оранжевый',
 'olive': 'оливковый',
 'navy': 'navy',
 'magenta': 'пурпурный',
 'linen': 'linen',
 'lime': 'лаймовый',
 'lilac': 'сиреневый',
 'lemon': 'lemon',
 'lavender': 'лавандовый',
 'khaki': 'хаки',
 'jade': 'нефритовый',
 'ivory': 'ivory',
 'indigo': 'индиго',
 'grey': 'серый',
 'green': 'зеленый',
 'gray': 'серый',
 'gold': 'золотой',
 'fuchsia': 'фуксия',
 'flax': 'flax',
 'emerald': 'emerald',
 'denim': 'denim',
 'cyan': 'бирюзовый',
 'cream': 'кремовый',
 'corn': 'corn',
 'coral': 'коралловый',
 'copper': 'медный',
 'cobalt': 'кобальтовый',
 'chocolate': 'шоколадный',
 'burgundy': 'бордовый',
 'buff': 'buff',
 'brown': 'коричневый',
 'bronze': 'бронзовый',
 'brass': 'латунный',
 'blue': 'голубой',
 'blond': 'blond',
 'black': 'черный',
 'beige': 'бежевый',
 'azure': 'лазурный',
 'aquamarine': 'аквамариновый',
 'aqua': 'аквамариновый',
 'amethyst': 'аметистовый',
 'amber': 'янтарный'
}

In [ ]:
color_vocab = {}
for color, v in colors_mapper.items():
    color_vocab[v] = len(color_vocab) + 1

In [ ]:
class Args:
    batch_size = 96
    epochs = 10
    lr = 1e-5

args = Args()

In [ ]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.pairs_len = len(self.pairs)

        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']

        categories = data['categories'].copy().apply(lambda x: x['3'])
        categories[~categories.isin(categories_map)] = 'rest'
        self.categories = categories.apply(lambda v: categories_map[v])

        def color_to_idx(colors):
            if colors is None:
                return []
            return [color_vocab[colors_mapper[color]] for color in colors]
        def drop_dup_colors(colors):
            if colors is None:
                return []
            res = []
            for v in colors:
                if v not in res:
                    res.append(v)
            return res
        colors = data['color_parsed'].copy().apply(color_to_idx).apply(drop_dup_colors)
        def pad_colors(colors):
            max_colors = 17
            if len(colors) > max_colors:
                return colors[:max_colors]
            return colors + [0] * (max_colors - len(colors))
        self.colors = colors.apply(pad_colors)

        self.names = data['name'].apply(text_preprocess)

        self.name_bert_embs = data['name_bert_64']

    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            self.categories[id1],
            torch.tensor(self.colors[id1]),
            self.names[id1],
            torch.tensor(self.main_pic_embs[id1]),
            torch.tensor(self.name_bert_embs[id1]),

            self.categories[id2],
            torch.tensor(self.colors[id2]),
            self.names[id2],
            self.main_pic_embs[id2],
            torch.tensor(self.name_bert_embs[id2]),

            target
        )

In [ ]:
class MultiModalNet(pl.LightningModule):
    margin = 0.75

    def __init__(self):
        super(MultiModalNet, self).__init__()

        # attrs
        self.category_embedding = nn.Embedding(
            num_embeddings=len(categories_map),
            embedding_dim=len(categories_map) // 2,
            padding_idx=None
        )

        self.color_embedding = nn.Embedding(
            num_embeddings=len(color_vocab) + 2,
            embedding_dim=(len(color_vocab) + 2) // 2,
            padding_idx=0
        )
        self.color_lstm_hidden = 64
        self.color_lstm = nn.LSTM(
            input_size=(len(color_vocab) + 2) // 2,
            hidden_size=self.color_lstm_hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        # name
        self.LaBSE_tokenizer = AutoTokenizer.from_pretrained('cointegrated/LaBSE-en-ru')
        self.LaBSE_model = AutoModel.from_pretrained('cointegrated/LaBSE-en-ru')
        self.LaBSE_fc = nn.Linear(768, 768)

        # net
        input_size = len(categories_map) // 2 + 2*self.color_lstm_hidden + 768 + 128 + 64
        output_size = 768
        self.bn = nn.BatchNorm1d(input_size)
        self.embedding_dropout = nn.Dropout(p=0.05)

        deberta_cfg = DebertaV2Config(
            hidden_size=input_size,
            num_hidden_layers=1,
            num_attention_heads=1,
            intermediate_size=1024,
        )
        self.deberta = DebertaV2Model(deberta_cfg, ).encoder

        features_num = 2 * input_size
        embedding_size = (features_num + output_size) // 2
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )

        self.output_layer = nn.Linear(embedding_size, output_size)

    def forward(self, categories, colors, names, pic_embs, name_bert_embs):
        categories_output = self.category_embedding(categories)

        colors_emb = self.color_embedding(colors)
        output, (ht, ct) = self.color_lstm(colors_emb)
        out_forward = output[:, -1, :self.color_lstm_hidden]
        out_reverse = output[:, 0, self.color_lstm_hidden:]
        colors_output = torch.cat([out_forward, out_reverse], 1)

        encoded_input = self.LaBSE_tokenizer(
            names, padding=True, truncation=True, max_length=256, return_tensors='pt'
        ).to('cuda')
        model_output = self.LaBSE_model(**encoded_input)
        embeddings = torch.nn.functional.normalize(model_output.pooler_output)
        names_output = self.LaBSE_fc(embeddings)

        pics_output = torch.nn.functional.normalize(pic_embs)

        names_bert_output = torch.nn.functional.normalize(name_bert_embs)

        x = torch.cat([categories_output, colors_output, names_output, pics_output, names_bert_output], dim=1)
        x = self.bn(x)
        x = self.embedding_dropout(x)
        x = x.unsqueeze(1)
        attention_mask = torch.ones((x.shape[0], 1), device='cuda')
        last_hidden = self.deberta(x, attention_mask)
        last_hidden = torch.concat([last_hidden[0].mean(1), last_hidden[0].max(1)[0]], -1)
        outputs = self.neck(last_hidden)
        outputs = self.output_layer(outputs)
        outputs = torch.nn.functional.normalize(outputs)
        return outputs


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=0.05
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        # self.log('step', batch_idx, logger=True, on_epoch=True)
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)

        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, on_step=False, logger=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)

        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=False, on_epoch=True, prog_bar=True)

        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0

        self.log("val_auc", auc, logger=False, on_epoch=True, prog_bar=True)

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

    def predict_step(self, batch, batch_idx):
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)

        dists = nn.PairwiseDistance()(out1, out2)
        return torch.cat([out1, out2, (1 - dists).unsqueeze(-1)], dim=1).detach().cpu()

In [ ]:
model = MultiModalNet()

In [ ]:
checkpoint_cb = ModelCheckpoint(
    dirpath='./MultiModal/', filename='products-{epoch:02d}-{val_auc:.4f}-normalize', monitor='val_auc', mode='max'
)

trainer = pl.Trainer(
    logger=False, # CSVLogger('./'),
    enable_checkpointing=True,
    callbacks=[checkpoint_cb],
    accelerator='gpu',
    devices=[0],
    profiler='advanced',
    precision="16-mixed",
    check_val_every_n_epoch=1,
    max_epochs=args.epochs
)

In [ ]:
test_pairs['target'] = -1
test_pairs = test_pairs[['target', 'variantid1', 'variantid2']]
test_dataset = ItemsDataset(test_pairs, test_data)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [ ]:
test_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, test_loader)])

In [ ]:
test_embeds = pd.Series(index=test_data.index, dtype='object', name='multimodal_tuned_768')
test_embeds[test_pairs.variantid1] = list(test_features[:, :768])
test_embeds[test_pairs.variantid2] = list(test_features[:, 768:768*2])
test_embeds